In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.getcwd()

'/content/drive/My Drive'

In [ ]:
os.listdir()

In [ ]:
%cd drive/MyDrive/

[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive


In [ ]:
!pip install -q -U bitsandbytes
!pip install datasets -U
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install pandas
!pip install wandb
!pip install lomo-optim

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# 토크나이저 및 데이터 준비

## Text Data 준비

In [ ]:
from datasets import load_dataset
import pandas as pd
from huggingface_hub import login

In [ ]:
# 허깅페이스 로그인 방법
my_hf_key='your api key'
login(my_hf_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# 모델 레포지토리
model_path = "beomi/Llama-3-Open-Ko-8B"

# 데이터 path
data_path = 'DopeorNope/Ko-Optimize_Dataset'

In [ ]:
# dataset 다운

data = load_dataset(data_path)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 10000
    })
})

In [ ]:
df = pd.DataFrame(data['train'])

In [ ]:
# 데이터셋 구성확인
df.head()

,input,instruction,output
0,,"∫[0,t] (GCP × SM × TA) dt = SA\n\n여기서\n∫ = 적분 ...","예, 주어진 공식을 다음과 같이 수정할 수 있습니다:\n\n∫[0,T] [(GCP ..."
1,귀하는 사람들이 정보를 찾도록 도와주는 AI 어시스턴트입니다. 사용자가 질문을 합니...,정답 -> 문장 A\n질문 -> 다음 두 문장 중 상식에 어긋나는 것은 어느 것입니...,질문 -> 다음 두 문장 중 상식에 어긋나는 것은 어느 것입니까?\n옵션:\n- 문...
2,,이산화탄소 배출을 줄이는 5가지 방법을 나열하세요.,"1. 재생 에너지원 사용: 태양열, 풍력, 수력, 지열 등 이산화탄소를 배출하지 않..."
3,귀하는 사람들이 정보를 찾도록 도와주는 AI 어시스턴트입니다. 사용자가 질문을 합니...,"많은 정치인들이 소농에 대해 이야기하는 것을 좋아하지만, 실제로 거의 모든 농장은 ...",서서히 해봅시다: 노화된 록스타의 손이 협조하지 않는 것은 건강 문제를 나타냅니다....
4,귀하는 항상 설명을 제공하는 도움이 되는 조수입니다. 5살짜리 아이에게 대답한다고 ...,다음 리뷰의 감상을 알려주세요: 솔직히 저는 사춘기 욕구 때문에 이 영화를 봤어요....,이 리뷰에는 긍정적인 감정이 담겨 있습니다. 영화를 보고 유머와 연기를 즐기며 즐거...


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import bitsandbytes as bnb
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# 토크나이저 세팅: QLoRA시 pad 토큰을 eos로 설정해주기
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})



tokenizer.padding_side = 'right' # padding to right (otherwise SFTTrainer shows warning)

pad = tokenizer.pad_token_id


tokenizer.pad_token_id = eos

tokenizer.padding_side = "right"

cut_off_len = 4098
val_size = 0.005
train_on_inputs = False
add_eos_token = False

In [ ]:
template = {
    "prompt_input": "아래는 문제를 설명하는 지시사항과, 구체적인 답변을 방식을 요구하는 입력이 함께 있는 문장입니다. 이 요청에 대해 적절하게 답변해주세요.\n###입력:{input}\n###지시사항:{instruction}\n###답변:",
    "prompt_no_input": "아래는 문제를 설명하는 지시사항입니다. 이 요청에 대해 적절하게 답변해주세요.\n###지시사항:{instruction}\n###답변:"
}

In [ ]:

from typing import Union

def generate_prompt(
    instruction: str,
    input: Union[None, str] = None,
    label: Union[None, str] = None,
    verbose: bool = False
) -> str:
    """
    주어진 instruction, input, label을 사용하여 프롬프트를 생성하는 함수.

    Parameters:
    - instruction (str): 문제 설명 또는 지시사항.
    - template (dict): 입력이 있는 경우와 없는 경우의 템플릿을 포함한 딕셔너리.
    - input (str or None): 문제에 대한 구체적인 입력 (옵션).
    - label (str or None): 정답 또는 응답 (옵션).
    - verbose (bool): 생성된 프롬프트를 출력할지 여부.

    Returns:
    - str: 완성된 프롬프트.
    """
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)

    if label:
        res = f"{res}{label}"

    if verbose:
        print(res)

    return res


In [ ]:
def tokenize(prompt, add_eos_token=True):
   result = tokenizer(prompt,truncation=True,max_length=cut_off_len,padding=False,return_tensors=None,)
   if (result["input_ids"][-1] != tokenizer.eos_token_id
       and len(result["input_ids"]) < cut_off_len
       and add_eos_token
      ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

   result["labels"] = result["input_ids"].copy()
   return result

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"]
        )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = generate_prompt(data_point["instruction"], data_point["input"])
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1



        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][user_prompt_len:]

    return tokenized_full_prompt

In [ ]:
if val_size > 0:
  train_val = data["train"].train_test_split(test_size=val_size, shuffle=True, seed=42)
  train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
  val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))
else:
  train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
  val_data = None

Map:   0%|          | 0/9950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

# Model 준비

In [ ]:
# Quantization config 준비

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16,
    )





In [ ]:
# Model 로드 하기
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    #quantization_config = quantization_config,
    torch_dtype = torch.bfloat16,
    device_map = {"" : 0}
    )


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# model.config.pad_token_id = tokenizer.pad_token_id # updating model config

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r = 16,
    lora_alpha =  ,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
    )

In [ ]:
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
  return list(lora_module_names)

In [ ]:
print('Trainable targer module:',find_all_linear_names(model))

Trainable targer module: ['o_proj', 'gate_proj', 'down_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj']


In [ ]:
# QLoRA 준비
model = get_peft_model(model, config)

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# 파라미터 수 체크
print_trainable_parameters(model)

trainable params: 8030261248 || all params: 8030261248 || trainable%: 100.0


In [ ]:
#Hyper parameter setting

output_dir='./llama_singleGPU-v1'


num_epochs = 1
micro_batch_size = 1
gradient_accumulation_steps = 16
warmup_steps = 10
learning_rate = 5e-7

group_by_length = False



optimizer = 'adalomo' #'paged_adamw_8bit'

# adam 활용시
beta1 = 0.9
beta2 = 0.95

lr_scheduler = 'cosine'
logging_steps = 1

use_wandb = True
wandb_run_name = 'Single_GPU_Optim'

use_fp16 = False
use_bf_16 = True
evaluation_strategy = 'steps'
eval_steps = 50
save_steps = 50
save_strategy = 'steps'




In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=TrainingArguments(
    per_device_train_batch_size = micro_batch_size,
    per_device_eval_batch_size = micro_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    num_train_epochs = num_epochs,
    learning_rate = learning_rate,
    adam_beta1 = beta1, # adam 활용할때 사용
    adam_beta2 = beta2, # adam 활용할때 사용
    fp16 = use_fp16,
    bf16 = use_bf_16,
    logging_steps = logging_steps,
    optim = optimizer,
    evaluation_strategy = evaluation_strategy if val_size > 0 else "no",
    save_strategy="steps",  #스텝기준으로 save
    eval_steps = eval_steps if val_size > 0 else None,
    save_steps = save_steps,
    lr_scheduler_type=lr_scheduler,
    output_dir = output_dir,
    load_best_model_at_end = True if val_size > 0 else False ,
    group_by_length=group_by_length,
    report_to="wandb" if use_wandb else None,
    run_name=wandb_run_name if use_wandb else None,
    ),
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
    )

In [ ]:
model.config.use_cache = False


trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
50,23.555700,1.406021
100,19.547800,1.406592


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
50,23.555700,1.406021
100,19.547800,1.406592


In [ ]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)


('./llama_singleGPU-v1/tokenizer_config.json',
 './llama_singleGPU-v1/special_tokens_map.json',
 './llama_singleGPU-v1/tokenizer.json')

In [ ]:
from peft import PeftModel


base_model= AutoModelForCausalLM.from_pretrained(model_path, token= my_hf_key)


merged_model= PeftModel.from_pretrained(base_model, output_dir)


merged_model= merged_model.merge_and_unload()

merged_model.push_to_hub('DopeorNope/Single_GPU_Llama3-8B')
tokenizer.push_to_hub('DopeorNope/Single_GPU_Llama3-8B')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

...

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DopeorNope/Single_GPU_Llama3-8B/commit/cc811947f458d8a2b49b1dff68300835c2103d71', commit_message='Upload tokenizer', commit_description='', oid='cc811947f458d8a2b49b1dff68300835c2103d71', pr_url=None, pr_revision=None, pr_num=None)